<a href="https://colab.research.google.com/github/farouk-naseri/PyProject/blob/main/Project_tp_Py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Generate random monthly sales data
np.random.seed(42)
months = pd.date_range("2025-01-01", "2025-12-01", freq="MS").strftime("%B")
data = {
    "Month": months,
    "Product_A": np.random.randint(50, 101, size=12),
    "Product_B": np.random.randint(30, 81, size=12),
    "Product_C": np.random.randint(20, 61, size=12),
    "Product_D": np.random.randint(10, 51, size=12),
}
df = pd.DataFrame(data)

# 2. Save raw data
df.to_csv("initial.csv", index=False)

# 3. Compute metrics
df["Total_Sales"] = df[["Product_A", "Product_B", "Product_C", "Product_D"]].sum(axis=1)
df["Average_Sales"] = df[["Product_A", "Product_B", "Product_C", "Product_D"]].mean(axis=1)
df["Month_over_Month_Growth_%"] = df["Total_Sales"].pct_change().fillna(0) * 100
quarters = {1: "Q1", 2: "Q1", 3: "Q1", 4: "Q2", 5: "Q2", 6: "Q2",
            7: "Q3", 8: "Q3", 9: "Q3", 10: "Q4", 11: "Q4", 12: "Q4"}
df["Quarter"] = pd.Series(range(1, 13)).map(quarters)
df["Max_Sales_Product"] = df[["Product_A", "Product_B", "Product_C", "Product_D"]].idxmax(axis=1)
df["Min_Sales_Product"] = df[["Product_A", "Product_B", "Product_C", "Product_D"]].idxmin(axis=1)

# 4. Save updated data
df.to_csv("final.csv", index=False)

# 5. Create pivot tables
pivot_avg = df.groupby("Quarter")[["Product_A", "Product_B", "Product_C", "Product_D"]].mean()
pivot_total = df.groupby("Quarter")["Total_Sales"].sum()
output = pd.concat([pivot_avg, pivot_total], axis=1)
output.to_csv("output.csv")

# 7. Visualizations
plt.figure(figsize=(10,6))
for product in ["Product_A", "Product_B", "Product_C", "Product_D"]:
    plt.plot(df["Month"], df[product], marker="o", label=product)
plt.title("Monthly Sales Line Chart")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.savefig("line_chart.png")
plt.close()

df.set_index("Month")[["Product_A","Product_B","Product_C","Product_D"]].plot(
    kind="bar", stacked=True, figsize=(10,6))
best_month = df.loc[df["Total_Sales"].idxmax(),"Month"]
plt.title(f"Stacked Bar Chart (Best Month: {best_month})")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("stacked_bar.png")
plt.close()

plt.figure(figsize=(10,6))
sns.heatmap(df.set_index("Month")[["Product_A","Product_B","Product_C","Product_D"]],
            annot=True, cmap="YlGnBu", fmt="d")
plt.title("Heatmap of Monthly Sales")
plt.tight_layout()
plt.savefig("heatmap.png")
plt.close()

plt.figure(figsize=(8,6))
sns.boxplot(data=df[["Product_A","Product_B","Product_C","Product_D"]])
plt.title("Boxplot of Sales Distribution per Product")
plt.tight_layout()
plt.savefig("boxplot.png")
plt.close()

# 8. Identify best month, product, and quarter
best_month = df.loc[df["Total_Sales"].idxmax(), "Month"]
best_product = df.drop(columns=["Month","Total_Sales","Average_Sales",
                                "Month_over_Month_Growth_%","Quarter",
                                "Max_Sales_Product","Min_Sales_Product"]).sum().idxmax()
best_quarter = pivot_total.idxmax()

# 9. Summary answers
summary = {
    "Best Month": best_month,
    "Best Product": best_product,
    "Best Quarter": best_quarter,
    "Top Contributing Product": best_product,
    "Quarter Performance": f"{best_quarter} performs best due to highest total sales.",
    "Recommendations": "Focus marketing on best product and quarter, expand distribution, and optimize weaker products."
}
summary_df = pd.DataFrame(list(summary.items()), columns=["Metric","Value"])
summary_df.to_csv("summary.csv", index=False)
